In [ ]:
from pyhive import hive
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import os
import time
from collections import Counter
import numpy as np

In [ ]:
# Read the data in from Hive and save to csv...if already have the csv, read it in

if 'time_series_data_reviews.csv' not in os.listdir():
    try:
        print('Getting data from Hive...')
        start = time.time()
        conn = hive.Connection(host = 'hadoop.rcc.uchicago.edu', 
                               port = 10000, 
                               username = 'mtoolsidas', 
                               database = 'dmp_yelp_rs')

        cursor = conn.cursor()

        query = 'select r.business_id,r.`date`,r.stars,b.simplified_category from review r join business_basicdata b on r.business_id = b.business_id'
        cursor.execute(query)
        res = cursor.fetchall()
        end = time.time()
        df = pd.DataFrame(res, columns = [i[0] for i in cursor.description])
        df.to_csv('time_series_data_reviews.csv', index = False)
        print('Finished.')
        print('Fetched data in {} seconds.'.format(end - start))
    except:
        print('ERROR: Connect to the VPN.')
        
else:
    print('Getting data from csv...')
    df = pd.read_csv('time_series_data_reviews.csv')
    print('Finished')

In [ ]:
# Checking data

df.head()

In [ ]:
# Cleaning the column names to remove '_.'

df.columns = [c[2:] for c in df.columns]

In [ ]:
# Converting `date` column to datetime and extracting date components

df['date'] = pd.to_datetime(df['date'], format = '%Y-%m-%d %H:%M:%S')
df['just_date'] = df['date'].dt.date
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['hour'] = df['date'].dt.hour
df['day_name'] = df['date'].dt.day_name()
df['month_year'] = df['date'].dt.to_period('M').astype(str)
df['week'] = df['date'].dt.to_period('W').astype(str)

In [ ]:
# Grouping data by category and month/year, getting the count of reviews and average stars for each group  

group_by_cat = df.groupby(['simplified_category','month_year']).agg({'business_id':'count', 'stars':'mean'})
group_by_cat = group_by_cat.rename(columns = {'business_id':'num_reviews'})
group_by_cat.index = group_by_cat.index.set_levels([group_by_cat.index.levels[0], pd.to_datetime(group_by_cat.index.levels[1])])

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

dfs = []

# Looping through each category (index level 0) and performing decomposition analysis
for i in set(group_by_cat.index.get_level_values(0)):
    
    print('starting category {}...'.format(i))
    
    # refactoring df to have 1 index instead of multi-index (removing category index)
    tmp = group_by_cat.xs(i, level = 0, drop_level = True)
    # changing index (month_year) to monthly period
    tmp.index = tmp.index.to_period('M')
    # filling in any missing months in the data and interpolating the values for review count and average stars
    tmp = tmp.reindex(pd.period_range(tmp.index[0],tmp.index[-1],freq='M')).interpolate('time')
    # turning index back to datetime
    tmp.index = tmp.index.to_timestamp()
    # Performing decomposition using additive model (as magnitude of seasonlity does not change over time)
    # This is done for both review count and average stars
    decompose_num_reviews = seasonal_decompose(tmp.num_reviews, model = 'additive', extrapolate_trend = 'freq')
    decompose_avg_stars = seasonal_decompose(tmp.stars, model = 'additive', extrapolate_trend = 'freq')
    # Turning result into dataframe and appending to list of dfs
    tmp2 = pd.DataFrame(decompose_num_reviews.seasonal)
    tmp2 = tmp2.rename(columns = {'seasonal':'seasonal_num_reviews'})
    tmp2['simplified_category'] = i
    tmp2['trend_num_reviews'] = decompose_num_reviews.trend
    tmp2['seasonal_avg_stars'] = decompose_avg_stars.seasonal
    tmp2['trend_avg_stars'] = decompose_avg_stars.trend
    dfs.append(tmp2)

    print('finished with category {}.'.format(i))
    print('--- --- --- ---')

In [ ]:
# Concatenating dataframes for all categories into one and checking data

final_df = pd.concat(dfs)
final_df = pd.concat(dfs).head()

In [ ]:
# Save result

final_df.to_csv('time_series_cat_final.csv', index = False)